# Run all the code cells to successfully make and start the PHP proxy

In [ ]:
#@title Run to setup PHP
!bash <(wget -O - https://raw.githubusercontent.com/Athlon1600/php-proxy-installer/master/install.sh)

In [2]:
#@title Run to install ngrok
!wget -q "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz" -O ngrok.gz
!gunzip "ngrok.gz"
!tar -xf ngrok
!mv ngrok /bin/ngrok

In [ ]:
#@title Run to add ngrok token
token = input("Enter ngrok token: ")
!ngrok authtoken {token} 2>&1

In [ ]:
#@title Run to install composer
!"$(php -r 'copy("https://composer.github.io/installer.sig", "php://stdout");')" > Echeck
!php -r "copy('https://getcomposer.org/installer', 'composer-setup.php');"
!"$(php -r "echo hash_file('sha384', 'composer-setup.php');")" > Acheck

with open("Echeck", "r") as f:
  Echeck = f.read()
with open("Acheck", "r") as f:
  Acheck = f.read()

if Echeck != Acheck:
  print('ERROR: Invalid installer checksum')
  !rm composer-setup.php

!php composer-setup.php 2>&1
!php -r "unlink('composer-setup.php');" 2>&1
!mv composer.phar /bin/composer 2>&1

In [ ]:
#@title Run to install the proxy
!composer create-project athlon1600/php-proxy-app:dev-master 2>&1

In [6]:
#@title Run to setup proxy
with open('php-proxy-app/config.php', 'r') as f:
    data = f.read().split('\n')

newdat = ''

for line in data:
    if line.startswith("$config['app_key'] = '';"):
        newdat += "$config['app_key'] = 'key12345';\n"
    else:
        newdat += line+'\n'

with open('php-proxy-app/config.php', 'w') as f:
    f.write(newdat)

In [ ]:
#@title Run to tidy up proxy files
%cd php-proxy-app
!wget -q "https://findicons.com/icon/download/direct/169016/php/128/ico" -O favicon.ico
!chmod +rwx -R .

In [ ]:
#@title Run to start proxy
!rm -rf ngrok.log

import threading
import time
import os
import re

ngrok = "ngrok tcp 8001 -log=stdout > ngrok.log"
php = "php -S localhost:8001"

Ngrok = threading.Thread(None, os.system, args=(ngrok,))
Php = threading.Thread(None, os.system, args=(php,))

# Run threads

Ngrok.start()
Php.start()

time.sleep(1.5)

with open("ngrok.log", 'r') as f:
    data = f.read()

# Regex to find URL

url = re.search("tcp.*$", data).group()

# Turn tcp to http

url = url.replace('tcp', 'http')

print(url)